# PSPNet을 활용한 물체 감지 흐름

PSPNet을 활용한 시맨틱 분할의 4단계 흐름:

1. 1단계에서는 전처리로 화상 크기를 475 * 475 픽셀로 리사이즈하고 색상 정보를 표준화 한다.

2. 2단계에서는 PSPNet 신경망에 전처리한 화상을 입력한다. PSPNet 출력으로 21by475by475(클래스 수, 높이, 폭)의 배열이 출력된다. 출력 배열의 값은 픽셀이 해당 클래스일 신뢰도에 대응한 값이다.

3. PSPNet 출력 값에 픽셀별로 신뢰도가 가장 높은 클래스와 각 픽셀이 대응할 것으로 예상되는 클래스를 구한다. 픽셀별 신뢰도가 최고로 높은 클래스 정보가 시맨틱 분할의 출력이 된다.

4. 4단계에서는 시맨틱 분할의 출력을 입력 화상의 원 크기로 리사이즈한다.

# DataLoader

In [ ]:
import os.path as osp
from PIL import Image

import torch.utils.data as data

In [ ]:
def make_datapath_list(rootpath):

    """
    학습 및 검증용 화상 데이터와 어노테이션 데이터의 파일 경로 리스트 작성
    """

    # 화상 파일과 어노테이션 파일의 경로 템플릿 작성
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
    annopath_template = osp.join(rootpath, 'SegmentationClass', '%s.png')

    # 훈련 및 검증 파일 각각의 ID 취득
    train_id_names = osp.join(rootpath + 'ImageSets/Segmentation/train.txt')
    val_id_names = osp.join(rootpath + 'ImageSets/Segmentation/val.txt')

    #훈련 데이터의 화상 파일과 어노테이션 파일의 경로 리스트 작성
    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)

    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
    
    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [ ]:
rootpath = "./data/VOCdevkit/VOC2012/"

train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

print(train_img_list[0])
print(train_anno_list[0])

Dataset 클래스를 작성하기 전 화상과 어노테이션을 전처리하는 DataTransform 클래스를 만든다.

In [ ]:
from utils.data_augumentation import Compose, Scale, RandomRotation, RandomMirror, Resize, Normalize_Tensor 


class DataTransform():

    def __init__(self, input_size, color_mean, color_std):
        self.data_transform = {
            "train": Compose([
                Scale(scale=[0.5, 1.5]),  
                RandomRotation(angle=[-10, 10]),  
                RandomMirror(), 
                Resize(input_size),  
                Normalize_Tensor(color_mean, color_std) 
            ]),
            'val': Compose([
                Resize(input_size),  
                Normalize_Tensor(color_mean, color_std)  
            ])
        }
    
    def __call__(self, phase, img, anno_class_img):

        return self.data_transform[phase](img, anno_class_img)

Dataset 클래스인 VOCDataset 클래스를 작성한다. VOCDataset 인스턴스 생성 시 화상 데이터  리스트, 어노테이션 데이터 리스트, 학습인지 검증인지 나타내는 phase 변수, 그리고 전처리 클래스의 인스턴스를 인수로 받는다. 

In [ ]:
class VOCDataset(data.Dataset):
    

    def __init__(self, img_list, anno_list, phase, transform):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
       
        img, anno_class_img = self.pull_item(index)
        return img, anno_class_img

    def pull_item(self, index):

        image_file_path = self.img_list[index]
        img = Image.open(image_file_path)   

        anno_file_path = self.anno_list[index]
        anno_class_img = Image.open(anno_file_path)  

        img, anno_class_img = self.transform(self.phase, img, anno_class_img)

        return img, anno_class_img


In [ ]:
# RGB 색의 평균치와 표준편차
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

# 데이터 셋 작성
train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

print(val_dataset.__getitem__(0)[0].shape)
print(val_dataset.__getitem__(0)[1].shape)
print(val_dataset.__getitem__(0))

DataLoader를 만든다. DataLoader 작성 방법은 Object Detection과 동일하다. 하지만 Object Detection과 달리 어노테이션 데이터 크기가 데이터마다 변하지 않아 파이토치의 DataLoader 클래스를 그대로 사용할 수 있다.

In [ ]:
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

batch_iterator = iter(dataloaders_dict["val"]) 
imges, anno_class_imges = next(batch_iterator)  
print(imges.size()) 
print(anno_class_imges.size())  